In [6]:
from stepmix.stepmix import StepMix
from stepmix.datasets import data_bakk_response, data_bakk_covariate
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.metrics import rand_score
from sklearn.model_selection import GridSearchCV, ParameterGrid
from stepmix.utils import get_mixed_descriptor
from scipy.stats import norm

X, Y, target = data_bakk_covariate(n_samples=2000, sep_level=.9,
                                   random_state=42)

In [7]:
# On propose un modèle de mesure dichotomique avec une covariable catégorielle
model = StepMix(n_components=3, n_steps=1, measurement='bernoulli',
                structural='covariate', random_state=42, verbose=0,
                max_iter=2000)
model.fit(X, Y)

Fitting StepMix...


Initializations (n_init) : 100%|█| 1/1 [00:00<00:00, 14.18it/s, max_LL=-5.19e+3, max_avg_LL=-2.6


StepMix(max_iter=2000, n_components=3, random_state=42, structural='covariate')

In [8]:
###beta sans bootstrap ###
# On soustrait la **DEUXIÈME** colonne pour normaliser les résultats. 
param = model.get_parameters()
param['structural']

{'beta': array([[-3.29006846,  1.03822369],
        [ 0.44527222,  0.0118777 ],
        [ 2.84325019, -1.04863614]])}

In [9]:

BETA = param['structural']['beta']
BETA -= BETA [1, :].reshape((1, -1))
BETA

array([[-3.73534067,  1.02634599],
       [ 0.        ,  0.        ],
       [ 2.39797797, -1.06051385]])

## Bootstrap

In [5]:
# On effectue 1000 échantillons boostrap de nos valeurs.
bs_params = model.bootstrap_stats(X, Y, n_repetitions=1000)


Bootstrapping estimator...


Bootstrap Repetitions    : 100%|█| 1000/1000 [01:02<00:00, 15.91it/s, max_LL=-4.95e+3, median_LL
C:\Python311\Lib\site-packages\stepmix\stepmix.py:1173: FutureWarning: The provided callable <function mean at 0x0000016874FCF4C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  return pd.pivot_table(
C:\Python311\Lib\site-packages\stepmix\stepmix.py:1173: FutureWarning: The provided callable <function std at 0x0000016874FCF600> is currently using DataFrameGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'std' instead.
  return pd.pivot_table(
C:\Python311\Lib\site-packages\stepmix\stepmix.py:1173: FutureWarning: The provided callable <function mean at 0x0000016874FCF4C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current b

In [11]:
# j'extrait les 6 paramètres logit non-normalisés en une seule colonne.
logit_1col = bs_params['samples'].xs("structural").iloc[:, 0]
logit_1col # deux "types" de beta : 1) feature_0 ; 2) intercept

model_name  param  class_no  variable 
covariate   beta   0         feature_0    0.966764
                             feature_0    1.027093
                             feature_0    1.104779
                             feature_0    1.004954
                             feature_0    1.009361
                                            ...   
                   2         intercept    2.882235
                             intercept    2.805039
                             intercept    2.979661
                             intercept    2.797481
                             intercept    2.799218
Name: value, Length: 6000, dtype: float64

In [50]:
logit_3cols = logit_1col.values.reshape([6, 1000]).transpose()
logit_3cols = pd.DataFrame(logit_3cols).iloc[:,[1, 0, 3, 2, 5, 4]]
logit_ref = pd.concat([logit_3cols.iloc[:, [2, 3]], logit_3cols.iloc[:, [2, 3]], logit_3cols.iloc[:, [2, 3]]], axis = 1)
logit_ref.columns = [0, 1, 2, 3, 4, 5]
logit_3cols.columns = [0, 1, 2, 3, 4, 5]
logit_3cols = logit_3cols - logit_ref
# feature_0 = colonnes 1-2-3 
# intercept = colonnes 4-5-6

# On ne peut donc pas utiliser la fonction .sub(logit_3cols.iloc[:,0], axis=0)
  #car les bêtas de feature_0 et l'intercept ne sont pas séparés

In [11]:
# Betas normalisés obtenus sans bootstrap
BETA

array([[-3.73534067,  1.02634599],
       [ 0.        ,  0.        ],
       [ 2.39797797, -1.06051385]])

In [51]:
pd.DataFrame({"mean": logit_3cols.apply(np.mean, axis = 0),
              "se": logit_3cols.apply(np.std, axis = 0),
              "z": logit_3cols.apply(np.mean, axis = 0)/logit_3cols.apply(np.std, axis = 0),
              "p-value": 2*norm.cdf(-np.abs(logit_3cols.apply(np.mean, axis = 0)/logit_3cols.apply(np.std, axis = 0)))}).

,mean,se,z,p-value
0,-3.761997,0.258371,-14.560445,5.013054e-48
1,1.033944,0.065703,15.736675,8.478353e-56
2,0.000000,0.000000,NaN,NaN
3,0.000000,0.000000,NaN,NaN
4,2.405852,0.161580,14.889531,3.854654e-50
5,-1.063938,0.070439,-15.104410,1.514549e-51
